

"""
DESCRIPTION:
    This sample demonstrates how to use basic agent operations from
    the Azure Agents service using a synchronous client.

USAGE:
    python sample_agents_basics.py

    Before running the sample:

    pip install azure-ai-projects azure-identity

    Set this environment variables with your own values:
    PROJECT_CONNECTION_STRING - the Azure AI Project connection string, as found in your AI Foundry project.
"""


https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?view=azure-python-preview&tabs=python&pivots=code-example

In [ ]:
%pip install azure-ai-projects
%pip install azure-identity

In [67]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)
   

In [68]:
bing_connection = project_client.connections.get(
    connection_name="api"
)
conn_id = bing_connection.id

print(conn_id)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

/subscriptions/0ac2cbd1-7fd2-435c-a854-3a3335dcb184/resourceGroups/agent1/providers/Microsoft.MachineLearningServices/workspaces/project-demo-b42h/connections/api


In [69]:

# Create agent with the bing tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model="gpt-4o",
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"}
    )
    print(f"Created agent, ID: {agent.id}")
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="pls summarize msft results in this url",
    )
    print(f"Created message, ID: {message.id}")
    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Retrieve run step details to get Bing Search query link
    # To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
    run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
    run_steps_data = run_steps['data']
    print(f"Last run step detail: {run_steps_data}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")


Created agent, ID: asst_5Oo5JiTwNZNwnpVk7hWYMR5D
Created thread, ID: thread_Oj3oH4xzf9EUruMXxyR7fczJ
Created message, ID: msg_V8e6TGKqzSwA1CfJbkSfW4gH
Run finished with status: RunStatus.COMPLETED
Last run step detail: [{'id': 'step_mriWFdPSqNeIMCU0LtdL8TeL', 'object': 'thread.run.step', 'created_at': 1737532242, 'run_id': 'run_7p9Zo5LP9iJSttTtRxpKpeOj', 'assistant_id': 'asst_5Oo5JiTwNZNwnpVk7hWYMR5D', 'thread_id': 'thread_Oj3oH4xzf9EUruMXxyR7fczJ', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1737532243, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_XTpXF95hFmzgQctZAqRfRUdk'}}, 'usage': {'prompt_tokens': 413, 'completion_tokens': 52, 'total_tokens': 465}}, {'id': 'step_xnUbQWjHBYi0Zvd1T95AEVAK', 'object': 'thread.run.step', 'created_at': 1737532241, 'run_id': 'run_7p9Zo5LP9iJSttTtRxpKpeOj', 'assistant_id': 'asst_5Oo5JiTwNZNwnpVk7hWYMR5D', 'threa

In [57]:
 # Delete the agent once done
project_client.agents.delete_agent("asst_0EuKisjjVWFGOBa2dOiNkkPx")
print("Deleted agent")

ResourceNotFoundError: (None) No assistant found with id 'asst_0EuKisjjVWFGOBa2dOiNkkPx'.
Code: None
Message: No assistant found with id 'asst_0EuKisjjVWFGOBa2dOiNkkPx'.

In [76]:
print(project_client.agents.delete_agent([project_client.agents.list_agents[0].id]))

TypeError: 'method' object is not subscriptable

In [77]:
print(project_client.agents.list_agents()[0].id)

ValueError: HTTP transport has already been closed. You may check if you're calling a function outside of the `with` of your client creation, or if you called `close()` on your client already.